# Installing the [Kaggle API](https://github.com/Kaggle/kaggle-api) in Colab

`kaggle` module should already be installed on Google Colab

In [ ]:
!pip install kaggle --upgrade

# Authenticating with Kaggle using kaggle.json

1. Navigate to https://www.kaggle.com. 
2. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) 
3. Select `Create API Token`. This will trigger the download of `kaggle.json`, a file containing your API credentials. If you open it, it'll look something like this: `{“username”:”YOUR-USER-NAME”,”key”:”SOMETHING-VERY-LONG”}`

Then run the cell below to upload kaggle.json to your Colab runtime.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Using the Kaggle API

For a more complete list of what you can do with the API, visit https://github.com/Kaggle/kaggle-api.

## Listing competitions

```
usage: kaggle competitions list [-h] [--group GROUP] [--category CATEGORY] [--sort-by SORT_BY] [-p PAGE] [-s SEARCH] [-v]

optional arguments:
  -h, --help            show this help message and exit
  --group GROUP         Search for competitions in a specific group. Default is 'general'. Valid options are 'general', 'entered', and 'inClass'
  --category CATEGORY   Search for competitions of a specific category. Default is 'all'. Valid options are 'all', 'featured', 'research', 'recruitment', 'gettingStarted', 'masters', and 'playground'
  --sort-by SORT_BY     Sort list results. Default is 'latestDeadline'. Valid options are 'grouped', 'prize', 'earliestDeadline', 'latestDeadline', 'numberOfTeams', and 'recentlyCreated'
  -p PAGE, --page PAGE  Page number for results paging. Page size is 20 by default 
  -s SEARCH, --search SEARCH
                        Term(s) to search for
  -v, --csv             Print results in CSV format
                        (if not set print in table format)

```

In [ ]:
!kaggle competitions list

In [ ]:
!kaggle competitions list -s titanic

## Downloading a dataset

```
usage: kaggle competitions download [-h] [-f FILE_NAME] [-p PATH] [-w] [-o]
                                    [-q]
                                    [competition]

optional arguments:
  -h, --help            show this help message and exit
  competition           Competition URL suffix (use "kaggle competitions list" to show options)
                        If empty, the default competition will be used (use "kaggle config set competition")"
  -f FILE_NAME, --file FILE_NAME
                        File name, all files downloaded if not provided
                        (use "kaggle competitions files -c <competition>" to show options)
  -p PATH, --path PATH  Folder where file(s) will be downloaded, defaults to current working directory
  -w, --wp              Download files to current working path
  -o, --force           Skip check whether local version of file is up to date, force file download
  -q, --quiet           Suppress printing information about the upload/download progress
```

> Note: you will need to accept competition rules at https://www.kaggle.com/c/ml-challenge-week6/rules

In [ ]:
!kaggle competitions download -c ml-challenge-week6

In [ ]:
!unzip census-income.data.zip

In [ ]:
import pandas as pd 

train_df = pd.read_csv('census-income.data', header=None)
train_df.info()

## Making a submission

In [ ]:
import numpy as np

submission_df = pd.read_csv('sampleSubmission.csv')
submission_df['income class'] = np.random.randint(0, 2, submission_df.shape[0])
submission_df['income class'].head()

In [ ]:
submission_df.to_csv('submission.csv')

```
usage: kaggle competitions submit [-h] -f FILE_NAME -m MESSAGE [-q]
                                  [competition]

required arguments:
  -f FILE_NAME, --file FILE_NAME
                        File for upload (full path)
  -m MESSAGE, --message MESSAGE
                        Message describing this submission

optional arguments:
  -h, --help            show this help message and exit
  competition           Competition URL suffix (use "kaggle competitions list" to show options)
                        If empty, the default competition will be used (use "kaggle config set competition")"
  -q, --quiet           Suppress printing information about the upload/download progress
```

In [ ]:
!kaggle competitions submit -c ml-challenge-week6 -f submission.csv -m "Test"

## List competition submissions

```
usage: kaggle competitions submissions [-h] [-v] [-q] [competition]

optional arguments:
  -h, --help   show this help message and exit
  competition  Competition URL suffix (use "kaggle competitions list" to show options)
               If empty, the default competition will be used (use "kaggle config set competition")"
  -v, --csv    Print results in CSV format (if not set print in table format)
  -q, --quiet  Suppress printing information about the upload/download progress
```

In [ ]:
!kaggle competitions submissions ml-challenge-week6

## Get competition leaderboard

```
usage: kaggle competitions leaderboard [-h] [-s] [-d] [-p PATH] [-v] [-q]
                                       [competition]

optional arguments:
  -h, --help            show this help message and exit
  competition           Competition URL suffix (use "kaggle competitions list" to show options)
                        If empty, the default competition will be used (use "kaggle config set competition")"
  -s, --show            Show the top of the leaderboard
  -d, --download        Download entire leaderboard
  -p PATH, --path PATH  Folder where file(s) will be downloaded, defaults to current working directory
  -v, --csv             Print results in CSV format (if not set print in table format)
  -q, --quiet           Suppress printing information about the upload/download progress
```

In [ ]:
!kaggle competitions leaderboard ml-challenge-week6 -s

## Uploading a Colab notebook to Kaggle Kernels

### Downloading a notebook from Colab

To download from Colab, use **File** | **Download .ipynb**

### Then upload the notebook to your Colab runtime

In [ ]:
uploaded = files.upload()
notebook_path = list(uploaded.keys())[0]

In [ ]:
!mkdir -p export
!mv "$notebook_path" export/

### Initialize metadata file for a kernel

```
usage: kaggle kernels init [-h] [-p FOLDER]

optional arguments:
  -h, --help            show this help message and exit
  -p FOLDER, --path FOLDER
                        Folder for upload, containing data files and a special kernel-metadata.json file       
                        (https://github.com/Kaggle/kaggle-api/wiki/Kernel-Metadata). 
                        Defaults to current working directory
```

In [ ]:
!kaggle kernels init -p export

In [ ]:
import re
your_kaggle_username = 'Your kaggle username'
notebook_title = 'New Kernel'
new_kernel_slug = re.sub(r'[^a-z0-9]+', '-', notebook_title.lower())
notebook_path = 'export'
metadata_file = 'export/kernel-metadata.json'

In [ ]:
import json

with open(metadata_file) as json_file:
    metadata = json.load(json_file)

metadata['code_file'] = notebook_path
metadata['kernel_type'] = 'notebook'
metadata['language'] = 'python'
metadata['id'] = f'{your_kaggle_username}/{new_kernel_slug}'
metadata['title'] = notebook_title

with open(metadata_file, 'w') as outfile:
    json.dump(metadata, outfile, indent=4)

In [ ]:
!cat export/kernel-metadata.json

```
usage: kaggle kernels push [-h] -p FOLDER

optional arguments:
  -h, --help            show this help message and exit
  -p FOLDER, --path FOLDER
                        Folder for upload, containing data files and a special kernel-metadata.json file 
                        (https://github.com/Kaggle/kaggle-api/wiki/Kernel-Metadata). 
                        Defaults to current working directory
```

In [ ]:
!kaggle kernels push -p export